In [ ]:
asin2 0.826

Outras similaridades: Cos, Ovl, Jacc, Dice

Semantic Similarity

These models find semantically similar sentences within one language or across languages:

**distiluse-base-multilingual-cased-v2**: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. While the original mUSE model only supports 16 languages, this multilingual knowledge distilled version supports 50+ languages.

**xlm-r-distilroberta-base-paraphrase-v1** - Multilingual version of distilroberta-base-paraphrase-v1, trained on parallel data for 50+ languages.

**xlm-r-bert-base-nli-stsb-mean-tokens**: Produces similar embeddings as the bert-base-nli-stsb-mean-token model. Trained on parallel data for 50+ languages.

**distilbert-multilingual-nli-stsb-quora-ranking** - Multilingual version of distilbert-base-nli-stsb-quora-ranking. Fine-tuned with parallel data for 50+ languages.

**T-Systems-onsite/cross-en-de-roberta-sentence-transformer** - Multilingual model for English an German. [More]

# Imports e métodos necessários

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import pandas as pd 
import xml.etree.ElementTree as et 

from scipy.stats import pearsonr

def parse_xml(xml_file):
    """Parse xml to pandas dataframe."""
    xtree = et.parse(xml_file)
    xroot = xtree.getroot() 

    df_cols = ['id', 't', 'h', 'similarity']
    rows = []

    for node in xroot:
        id_ = node.attrib.get("id")
        similarity = node.attrib.get("similarity")
        t = node.find("t").text
        h = node.find("h").text

        rows.append({
            "id": id_,
            "t": t, 
            "h": h,
            "similarity": similarity
        })
    return pd.DataFrame(rows, columns=df_cols, dtype=float)

def eval_similarity(pairs_gold, pairs_sys):
    '''
    Evaluate the semantic similarity output of the system against a gold score. 
    Results are printed to stdout.
    '''
    
    gold_values = np.array(pairs_gold)
    sys_values = np.array(pairs_sys)
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()
    
    print()
    print('Similarity evaluation')
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.3f}\t\t{:18.2f}'.format(pearson, mse))

# Carregando os dados

In [3]:
!ls ../data/assin2

assin2-blind-test.xml  assin2-dev.xml  assin2-test.xml	assin2-train-only.xml


In [4]:
df_ptbr_train = parse_xml('../data/assin2/assin2-train-only.xml')
df_ptbr_dev = parse_xml('../data/assin2/assin2-dev.xml')
df_ptbr_test = parse_xml('../data/assin2/assin2-test.xml')

In [5]:
print(f'assin-ptbr-train: {df_ptbr_train.shape}')
print(f'assin-ptbr-dev: {df_ptbr_dev.shape}')
print(f'assin-ptbr-test: {df_ptbr_test.shape}')

assin-ptbr-train: (6500, 4)
assin-ptbr-dev: (500, 4)
assin-ptbr-test: (2448, 4)


In [6]:
df_ptbr_train.head()

,id,t,h,similarity
0,1.0,Uma criança risonha está segurando uma pistola...,Uma criança está segurando uma pistola de água,4.5
1,2.0,Os homens estão cuidadosamente colocando as ma...,Os homens estão colocando bagagens dentro do p...,4.5
2,3.0,Uma pessoa tem cabelo loiro e esvoaçante e est...,Um guitarrista tem cabelo loiro e esvoaçante,4.7
3,4.0,Batatas estão sendo fatiadas por um homem,O homem está fatiando a batata,4.7
4,5.0,Um caminhão está descendo rapidamente um morro,Um caminhão está rapidamente descendo o morro,4.9


# Testes

## Sentence-BERT

### distiluse-base-multilingual-cased-v2

In [38]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [39]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [40]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [41]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [42]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.753		              0.55


In [43]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.705


### xlm-r-distilroberta-base-paraphrase-v1

In [9]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1', device='cpu')

In [17]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist(), show_progress_bar=True)
h_embeddings = model.encode(df_ptbr_test['h'].tolist(), show_progress_bar=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

In [18]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [19]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [20]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.676		              1.24


In [14]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.723


### xlm-r-bert-base-nli-stsb-mean-tokens

In [13]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

In [14]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [15]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [16]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [17]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.794		              0.63


In [18]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.760


### distilbert-multilingual-nli-stsb-quora-ranking

In [19]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [20]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [21]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [22]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [23]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.720		              1.59


In [24]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.677


### T-Systems-onsite/cross-en-de-roberta-sentence-transformer

In [31]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404


In [32]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [33]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [34]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [35]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.804		              0.64


In [36]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.747


## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### distiluse-base-multilingual-cased-v2

In [8]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '/mnt/data/sbert_finetuning_assin2_distiluse-base-multilingual-cased-v2'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)


model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


0.794293086543723

In [9]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [10]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [11]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [12]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.825		              0.48


In [13]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.794


### xlm-r-distilroberta-base-paraphrase-v1

In [31]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '../data/sbert_brt-pt'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)


# model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1', device='cpu')

pretrained_model = '../data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
word_embedding_model = models.Transformer(pretrained_model)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device='cpu')


train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

#model = SentenceTransformer(model_save_path)
#test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
#test_evaluator(model, output_path=model_save_path)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [32]:
model = SentenceTransformer(model_save_path, device='cpu')

In [33]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist(), show_progress_bar=True)
h_embeddings = model.encode(df_ptbr_test['h'].tolist(), show_progress_bar=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

In [34]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [35]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [36]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.828		              0.51


In [ ]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

In [37]:
df_ptbr_test['similarity_finetuning_bertpt'] = similarities
df_ptbr_test.head()

,id,t,h,similarity,similarity_pretrained,similarity_pretrained_bertpt,similarity_finetuningt,similarity_finetuning_bertpt
0,0.0,O cachorro caramelo está assistindo um cachorr...,Um cachorro de estimação está de pé no banco e...,3.80,3.923177,4.191451,3.870823,3.891498
1,1.0,O cara está fazendo exercícios no chão,Um cara está fazendo exercícios,3.75,4.495929,4.569058,4.401187,4.525919
2,2.0,Um cachorro grande e um cachorro pequenino est...,Um cachorro grande e um cachorro pequenino est...,4.40,4.947445,4.769739,4.815322,4.690919
3,3.0,Um menino jovem vestindo um traje de banho ver...,Um menino jovem está vestindo um maiô vermelho...,3.80,4.516025,4.694800,4.078489,4.295658
4,4.0,Um cara velho com uma barba que é cinza está a...,Um cara velho com uma barba grisalha está anda...,4.75,4.966382,4.725322,4.965317,4.872882


In [48]:
pd.set_option('display.max_colwidth', None)

df_ptbr_test['diff'] = abs(df_ptbr_test['similarity_pretrained'] - df_ptbr_test['similarity_finetuningt'])
df_ptbr_test.sort_values(by='diff', ascending=False).head(20)

,id,t,h,similarity,similarity_pretrained,similarity_pretrained_bertpt,similarity_finetuningt,similarity_finetuning_bertpt,diff
1391,1391.0,Uma senhora asiática está colocando maquiagem,A senhora está se maquiando,4.00,2.193500,4.080158,4.145394,4.427065,1.951894
1387,1387.0,A senhora está pegando o canguru,Um canguru está pegando o bebê da senhora,2.00,3.946067,4.493878,2.048501,2.032706,1.897566
2276,2276.0,O cara está tirando uma foto dele mesmo e de outro cara,Uma pessoa está segurando um dispositivo eletrônico,3.25,1.343563,3.260830,3.152490,2.763117,1.808927
2304,2304.0,Não tem nenhum praticante de snowboard dando um giro sobre um monte de neve,Alguém está pulando no ar em uma prancha,3.10,1.345685,3.709676,3.027666,3.036779,1.681981
1815,1815.0,Vários meninos estão pulando em um trampolim,Um homem está dando mortais em um trampolim,2.25,1.579163,4.034966,3.259903,2.862473,1.680741
528,528.0,O homem está fazendo exercício,Não tem nenhum homem fazendo levantamentos na barra,3.50,1.666560,3.565370,3.289549,3.756472,1.622989
261,261.0,Uma foto dele mesmo e outro cara está sendo tirada pelo homem,Um homem está segurando um aparelho eletrônico,2.20,1.544027,3.551390,3.144179,3.216529,1.600152
242,242.0,Um homem está colocando um dispositivo eletrônico,O homem está tirando uma foto dele mesmo e de outro cara,1.90,1.887419,3.552209,3.374093,3.136974,1.486674
1270,1270.0,A senhora está limpando um camarão,Alguém está limpando um animal,4.00,2.582048,4.219736,4.064645,4.197750,1.482597
524,524.0,Uma senhora está esfregando uma abobrinha com uma escova para vegetais,Uma pessoa está esfregando uma abobrinha,3.75,3.285456,4.295602,4.737941,4.748093,1.452485


### xlm-r-bert-base-nli-stsb-mean-tokens

In [20]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '/mnt/data/sbert_finetuning_assin2_xlm-r-bert-base-nli-stsb-mean-tokens'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)


model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

0.8116924067191124

In [21]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [22]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [23]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [24]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.840		              0.42


In [25]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.812


### distilbert-multilingual-nli-stsb-quora-ranking

In [26]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '/mnt/data/sbert_finetuning_assin2_distilbert-multilingual-nli-stsb-quora-ranking'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)


model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

0.7987335000703808

In [27]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [28]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [29]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [30]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.827		              0.48


In [31]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.799


### T-Systems-onsite/cross-en-de-roberta-sentence-transformer

In [34]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '/mnt/data/sbert_finetuning_assin2_T-Systems-onsite_cross-en-de-roberta-sentence-transformer'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)


model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404


0.8185809102687664

In [35]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [36]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [37]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [38]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.41


In [39]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.819
